In [ ]:
"""
A. Tham khảo hưỡng dẫn về GNN tại:
+ https://viblo.asia/p/gioi-thieu-ve-graph-neural-networks-gnns-yZjJYG7MVOE
+ https://docs.dgl.ai/en/0.8.x/tutorials/blitz/4_link_predict.html
+ https://arxiv.org/ftp/arxiv/papers/1812/1812.08434.pdf


B. Yêu cầu:
1. Tìm hiểu và trình bày tổng quan về GNN
2. Sử dụng GCN để dự đoán liên kết mạng xã hội mầ bạn lựa chọn
+ https://stellargraph.readthedocs.io/en/stable/demos/link-prediction/gcn-link-prediction.html
3. Đánh giá kết quả giữa GCN và các phương pháp trong LAB 04.01 / 04.02
"""

In [ ]:
# If you're using Google Colab:
!pip install torch-geometric
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}+${CUDA}.html

In [ ]:
import networkx as nx
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Traditional Link Prediction class (from Lab04_01)
class TraditionalLinkPrediction:
    def __init__(self, dataset='karate'):
        if dataset == 'karate':
            self.G = nx.karate_club_graph()
        elif dataset == 'les':
            self.G = nx.les_miserables_graph()
        else:
            raise ValueError("Dataset không hợp lệ")

    def evaluate_methods(self):
        """Tính toán các metrics cho phương pháp truyền thống."""
        edges = list(self.G.edges())
        np.random.seed(42)
        np.random.shuffle(edges)

        n_test = int(len(edges) * 0.2)
        test_edges = edges[:n_test]
        train_edges = edges[n_test:]

        # Create training graph
        train_G = self.G.copy()
        train_G.remove_edges_from(test_edges)

        # Get negative samples
        non_edges = list(nx.non_edges(train_G))
        np.random.shuffle(non_edges)
        test_non_edges = non_edges[:n_test]

        methods = {
            'Common Neighbors': lambda u, v: len(list(nx.common_neighbors(train_G, u, v))),
            'Jaccard Coefficient': lambda u, v: list(nx.jaccard_coefficient(train_G, [(u, v)]))[0][2],
            'Adamic-Adar': lambda u, v: list(nx.adamic_adar_index(train_G, [(u, v)]))[0][2]
        }

        results = []
        for name, score_func in methods.items():
            # Calculate scores
            pos_scores = [score_func(u, v) for u, v in test_edges]
            neg_scores = [score_func(u, v) for u, v in test_non_edges]

            # Prepare labels and predictions
            y_true = [1] * len(pos_scores) + [0] * len(neg_scores)
            y_scores = pos_scores + neg_scores

            # Use median as threshold
            threshold = np.median(y_scores)
            y_pred = [1 if score > threshold else 0 for score in y_scores]

            # Calculate metrics
            results.append({
                'Method': name,
                'AUC': roc_auc_score(y_true, y_scores),
                'Accuracy': accuracy_score(y_true, y_pred),
                'Precision': precision_score(y_true, y_pred),
                'Recall': recall_score(y_true, y_pred)
            })

        return pd.DataFrame(results)

# GCN Implementation (same as before)
class SimpleGCNConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(SimpleGCNConv, self).__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x, adj):
        deg = adj.sum(dim=1)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm_adj = deg_inv_sqrt.unsqueeze(-1) * adj * deg_inv_sqrt.unsqueeze(0)
        return torch.matmul(norm_adj, self.linear(x))

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = SimpleGCNConv(in_channels, hidden_channels)
        self.conv2 = SimpleGCNConv(hidden_channels, out_channels)

    def encode(self, x, adj):
        x = self.conv1(x, adj)
        x = F.relu(x)
        x = F.dropout(x, p=0.2, training=self.training)
        return self.conv2(x, adj)

    def decode(self, z, edge_index):
        return (z[edge_index[0]] * z[edge_index[1]]).sum(dim=-1)

    def forward(self, x, adj, edge_index):
        z = self.encode(x, adj)
        return self.decode(z, edge_index)

class LinkPredictionGCN:
    def __init__(self, dataset='karate'):
        if dataset == 'karate':
            self.G = nx.karate_club_graph()
        elif dataset == 'les':
            self.G = nx.les_miserables_graph()
        else:
            raise ValueError("Invalid dataset")

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def prepare_data(self):
        adj_matrix = nx.adjacency_matrix(self.G).todense()
        self.adj = torch.FloatTensor(adj_matrix).to(self.device)

        degrees = torch.tensor([[d] for n, d in self.G.degree()], dtype=torch.float)
        self.x = degrees.to(self.device)

        edges = list(self.G.edges())
        n_edges = len(edges)
        n_test = int(0.2 * n_edges)

        np.random.seed(42)
        np.random.shuffle(edges)

        self.train_edges = edges[n_test:]
        self.test_edges = edges[:n_test]

        non_edges = list(nx.non_edges(self.G))
        np.random.shuffle(non_edges)

        self.train_non_edges = non_edges[n_test:2*n_test]
        self.test_non_edges = non_edges[:n_test]

        return self.adj, self.x

    def train_model(self, epochs=100):
        model = GCN(in_channels=1, hidden_channels=64, out_channels=32).to(self.device)
        optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
        criterion = torch.nn.BCEWithLogitsLoss()

        self.model = model
        train_metrics = []

        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()

            train_edge_index = torch.tensor([[u, v] for u, v in self.train_edges + self.train_non_edges]).t().to(self.device)
            train_labels = torch.tensor([1] * len(self.train_edges) + [0] * len(self.train_non_edges)).float().to(self.device)

            out = model(self.x, self.adj, train_edge_index)
            loss = criterion(out, train_labels)

            loss.backward()
            optimizer.step()

            if (epoch + 1) % 10 == 0:
                model.eval()
                with torch.no_grad():
                    val_auc = self.evaluate()
                    train_metrics.append({
                        'epoch': epoch + 1,
                        'loss': loss.item(),
                        'val_auc': val_auc
                    })
                    print(f'Epoch: {epoch+1:03d}, Loss: {loss:.4f}, Val AUC: {val_auc:.4f}')

        return train_metrics

    def evaluate(self):
        self.model.eval()
        with torch.no_grad():
            test_edge_index = torch.tensor([[u, v] for u, v in self.test_edges + self.test_non_edges]).t().to(self.device)
            test_labels = torch.tensor([1] * len(self.test_edges) + [0] * len(self.test_non_edges))

            out = self.model(self.x, self.adj, test_edge_index)
            out = torch.sigmoid(out).cpu()

            return roc_auc_score(test_labels, out)

    def test_model(self):
        self.model.eval()
        with torch.no_grad():
            test_edge_index = torch.tensor([[u, v] for u, v in self.test_edges + self.test_non_edges]).t().to(self.device)
            test_labels = torch.tensor([1] * len(self.test_edges) + [0] * len(self.test_non_edges))

            out = self.model(self.x, self.adj, test_edge_index)
            out = torch.sigmoid(out).cpu()
            predictions = (out > 0.5).float()

            test_auc = roc_auc_score(test_labels, out)
            accuracy = accuracy_score(test_labels, predictions)
            precision = precision_score(test_labels, predictions)
            recall = recall_score(test_labels, predictions)

            return {
                'Method': 'GCN',
                'AUC': test_auc,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall
            }

def plot_comparison(traditional_results, gcn_results):
    # Combine results
    all_results = pd.concat([traditional_results, pd.DataFrame([gcn_results])], ignore_index=True)

    # Create bar plot
    metrics = ['AUC', 'Accuracy', 'Precision', 'Recall']
    plt.figure(figsize=(12, 6))

    x = np.arange(len(metrics))
    width = 0.15
    n_methods = len(all_results)

    for i, (idx, row) in enumerate(all_results.iterrows()):
        offset = width * (i - (n_methods-1)/2)
        plt.bar(x + offset, [row[metric] for metric in metrics], width, label=row['Method'])

    plt.xlabel('Metrics')
    plt.ylabel('Score')
    plt.title('Comparison of Link Prediction Methods')
    plt.xticks(x, metrics)
    plt.legend()
    plt.tight_layout()
    plt.show()

def compare_methods(dataset='karate'):
    # Run GCN
    print("Training GCN model...")
    gcn_predictor = LinkPredictionGCN(dataset)
    gcn_predictor.prepare_data()
    train_metrics = gcn_predictor.train_model()
    gcn_results = gcn_predictor.test_model()

    # Run traditional methods
    print("\nRunning traditional methods...")
    traditional_predictor = TraditionalLinkPrediction(dataset)
    traditional_results = traditional_predictor.evaluate_methods()

    # Plot comparison
    plot_comparison(traditional_results, gcn_results)

    # Print all results
    print("\nComplete Results:")
    all_results = pd.concat([traditional_results, pd.DataFrame([gcn_results])], ignore_index=True)
    print(all_results)


In [ ]:
# Run the comparison
if __name__ == "__main__":
    compare_methods('karate')

Ta có thể thấy GCN vượt trội đáng kể so với tất cả các phương pháp truyền thống với AUC là 0,818.

Recall 1,0 cho thấy phương pháp này xác định thành công tất cả các liên kết dương tính thực
AUC và Accuracy là 0,5 cho thấy phương pháp này đưa ra một số dự đoán dương tính giả

AUC cao nhưng độ chính xác vừa phải cho thấy khả năng xếp hạng tốt nhưng vẫn còn chỗ để tối ưu hóa ngưỡng

GCN cho thấy sự vượt trội rõ ràng với cải thiện ~30% về AUC so với phương pháp truyền thống tốt nhất

Khoảng cách đáng kể về Recall 1,0 và 0,4 cho thấy Khả năng tìm kết nối thực tế tốt hơn của GCN